Step 1: Install Required Libraries

In [1]:
pip install langchain openai faiss-cpu tiktoken gradio


  Using cached tiktoken-0.9.0-cp313-cp313-win_amd64.whl.metadata (6.8 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-win_amd64.whl.metadata (2.1 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.23.0-cp313-cp313-win_amd64.whl.metadata (3.0 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.10-py3-non

In [2]:
pip install transformers sentence-transformers faiss-cpu torch


  Using cached torch-2.7.0-cp313-cp313-win_amd64.whl.metadata (29 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.5 MB 3.8 MB/s eta 0:00:03
   --------- ------------------------------ 2.4/10.5 MB 6.7 MB/s eta 0:00:02
   ----------------- ---------------------- 4.5/10.5 MB 7.8 MB/s eta 0:00:01
   ------------------------ --------------- 6.3/10.5 MB 8.3 MB/s eta 0:00:01
   ------------------------------- -------- 8.1/10.5 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------  10.2/10.5 MB 8.8 MB/s eta 0:00:01
   ---------------------------------------- 10.5/10.5 MB 8.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ------------------------------ --------- 1.8/2.4 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 2.4/

Step 2: Load and Clean Your Text File
python
Copy code


In [3]:
import re

# Load the book
with open("atomic_habits.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Clean
cleaned_text = re.sub(r'\s+', ' ', text).strip()

print(cleaned_text[:1000])


AN IMPRINT OF PENGUIN RANDOM HOUSE LLC 375 Hudson Street New York, New York 10014 Copyright © 2018 by James Clear Penguin supports copyright. Copyright fuels creativity, encourages diverse voices, promotes free speech, and creates a vibrant culture. Thank you for buying an authorized edition of this book and for complying with copyright laws by not reproducing, scanning, or distributing any part of it in any form without permission. You are supporting writers and allowing Penguin to continue to publish books for every reader. Ebook ISBN 9780735211308 While the author has made every effort to provide accurate Internet addresses at the time of publication, neither the publisher nor the author assumes any responsibility for errors, or for changes that occur after publication. Further, the publisher does not have any control over and does not assume any responsibility for author or third-party websites or their content. Version_1 a·tom·ic əˈtämik 1. an extremely small amount of a thing; th

Step 3: Split the Text into Chunks

In [4]:
from langchain.text_splitter import CharacterTextSplitter

# Use simple text splitter
splitter = CharacterTextSplitter(
    separator=". ",
    chunk_size=500,
    chunk_overlap=50
)

chunks = splitter.split_text(cleaned_text)
print(f"Chunks: {len(chunks)}")


Created a chunk of size 1578, which is longer than the specified 500
Created a chunk of size 750, which is longer than the specified 500
Created a chunk of size 565, which is longer than the specified 500
Created a chunk of size 577, which is longer than the specified 500
Created a chunk of size 617, which is longer than the specified 500
Created a chunk of size 505, which is longer than the specified 500
Created a chunk of size 676, which is longer than the specified 500
Created a chunk of size 533, which is longer than the specified 500
Created a chunk of size 1734, which is longer than the specified 500
Created a chunk of size 2965, which is longer than the specified 500
Created a chunk of size 742, which is longer than the specified 500
Created a chunk of size 3219, which is longer than the specified 500
Created a chunk of size 1060, which is longer than the specified 500
Created a chunk of size 640, which is longer than the specified 500
Created a chunk of size 879, which is longe

Chunks: 1058


Step 4: Create Vector Store (FAISS)

In [6]:
import os
import certifi

# Set the environment variable to point to certifi's CA bundle
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()


In [7]:
import os
import certifi

# Fix SSL error for HuggingFace downloads
os.environ['REQUESTS_CA_BUNDLE'] = certifi.where()

from sentence_transformers import SentenceTransformer

# Now download and load the model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

print("✅ SentenceTransformer loaded successfully!")


c:\Users\Tousif M Tamboli\Downloads\llmProject\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Tousif M Tamboli\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is 

✅ SentenceTransformer loaded successfully!


In [8]:
# Assuming you already split your cleaned text into chunks called `chunks`
print(f"Number of chunks: {len(chunks)}")

# Generate embeddings for all chunks
embeddings = embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)

print(f"Embeddings shape: {embeddings.shape}")


Number of chunks: 1058


Batches: 100%|██████████| 34/34 [00:18<00:00,  1.87it/s]

Embeddings shape: (1058, 384)


In [9]:
import faiss

dimension = embeddings.shape[1]  # embedding size, e.g., 384 for all-MiniLM-L6-v2
index = faiss.IndexFlatL2(dimension)  # L2 distance

index.add(embeddings)
print(f"FAISS index contains {index.ntotal} vectors")


FAISS index contains 1058 vectors


In [10]:
def retrieve(query, top_k=3):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = [chunks[i] for i in indices[0]]
    return results


Step 7: Ask Questions!

In [11]:
query = "How can I break a bad habit?"
retrieved_chunks = retrieve(query)

for i, chunk in enumerate(retrieved_chunks, 1):
    print(f"--- Result {i} ---\n{chunk}\n")


--- Result 1 ---
When you forget to do a habit, make sure you get back on track immediately. HOW TO BREAK A BAD HABIT Inversion of the 1st Law: Make It Invisible 1.5: Reduce exposure. Remove the cues of your bad habits from your environment. Inversion of the 2nd Law: Make It Unattractive 2.4: Reframe your mind-set. Highlight the benefits of avoiding your bad habits. Inversion of the 3rd Law: Make It Difficult 3.6: Increase friction. Increase the number of steps between you and your bad habits

--- Result 2 ---
The key to finding and fixing the causes of your bad habits is to reframe the associations you have about them. It’s not easy, but if you can reprogram your predictions, you can transform a hard habit into an attractive one. Chapter Summary The inversion of the 2nd Law of Behavior Change is make it unattractive. Every behavior has a surface level craving and a deeper underlying motive. Your habits are modern-day solutions to ancient desires

--- Result 3 ---
Once a habit is forme

In [12]:
prompt = "What are the key steps to build a new habit successfully?"
retrieved_chunks = retrieve(prompt)

print(f"Prompt: {prompt}\n")
for i, chunk in enumerate(retrieved_chunks, 1):
    print(f"Result {i}:\n{chunk}\n")


Prompt: What are the key steps to build a new habit successfully?

Result 1:
Write down your current habits to become aware of them. 1.2: Use implementation intentions: “I will [BEHAVIOR] at [TIME] in [LOCATION].” 1.3: Use habit stacking: “After [CURRENT HABIT], I will [NEW HABIT].” 1.4: Design your environment. Make the cues of good habits obvious and visible. The 2nd Law: Make It Attractive 2.1: Use temptation bundling. Pair an action you want to do with an action you need to do. 2.2: Join a culture where your desired behavior is the normal behavior

Result 2:
Write down your current habits to become aware of them. 1.2: Use implementation intentions: “I will [BEHAVIOR] at [TIME] in [LOCATION].” 1.3: Use habit stacking: “After [CURRENT HABIT], I will [NEW HABIT].” 1.4: Design your environment. Make the cues of good habits obvious and visible. The 2nd Law:Make It Attractive 2.1: Use temptation bundling. Pair an action you want to do with an action you need to do. 2.2: Join a culture wh

In [14]:
prompt = "How can I stay motivated when trying to improve myself?"
retrieved_chunks = retrieve(prompt)

print(f"Prompt: {prompt}\n")
for i, chunk in enumerate(retrieved_chunks, 1):
    print(f"Result {i}:\n{chunk}\n")


Prompt: How can I stay motivated when trying to improve myself?

Result 1:
Even if I wasn’t motivated beforehand, by the time I was done with my ritual, I was in “game mode.” You can adapt this strategy for nearly any purpose. Say you want to feel happier in general. Find something that makes you truly happy—like petting your dog or taking a bubble bath—and then create a short routine that you perform every time before you do the thing you love. Maybe you take three deep breaths and smile. Three deep breaths. Smile. Pet the dog. Repeat

Result 2:
But if you maintain the same sloppy, pack-rat habits that led to a messy room in the first place, soon you’ll be looking at a new pile of clutter and hoping for another burst of motivation. You’re left chasing the same outcome because you never changed the system behind it. You treated a symptom without addressing the cause. Achieving a goal only changes your life for the moment. That’s the counterintuitive thing about improvement

Result 3:
H

In [16]:
prompt = "i have a habit of smoking how to get rid of it?"
retrieved_chunks = retrieve(prompt)

print(f"Prompt: {prompt}\n")
for i, chunk in enumerate(retrieved_chunks, 1):
    print(f"Result {i}:\n{chunk}\n")


Prompt: i have a habit of smoking how to get rid of it?

Result 1:
If you want to drink more water, fill up a few water bottles each morning and place them in common locations around the house. If you want to make a habit a big part of your life, make the cue a big part of your environment. The most persistent behaviors usually have multiple cues. Consider how many different ways a smoker could be prompted to pull out a cigarette: driving in the car, seeing a friend smoke, feeling stressed at work, and so on. The same strategy can be employed for good habits

Result 2:
You know you don’t actually want to smoke. You know you don’t really enjoy this.’ It helps you feel like you’re not the victim anymore. You start to realize that you don’t need to smoke.” I had never tried a cigarette, but I took a look at the book afterward out of curiosity. The author employs an interesting strategy to help smokers eliminate their cravings. He systematically reframes each cue associated with smoking an

Make the outputs more accurate!

In [17]:
def retrieve(query, top_k=5):  # increase from 3 to 5 or 10
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = [chunks[i] for i in indices[0]]
    return results


In [19]:
pip install transformers torch


Note: you may need to restart the kernel to use updated packages.


In [20]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load tokenizer and model (GPT-2 small)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Put model in eval mode and use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


c:\Users\Tousif M Tamboli\Downloads\llmProject\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Tousif M Tamboli\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regu

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [23]:
def generate_response(context_and_question, max_new_tokens=150):
    inputs = tokenizer.encode(context_and_question, return_tensors="pt").to(device)
    
    outputs = model.generate(
        inputs,
        max_new_tokens=max_new_tokens,  # generate this many new tokens only
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_p=0.9,
        top_k=50
    )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = generated_text[len(context_and_question):].strip()
    return answer


Testing Again

In [25]:
prompt = "I have a habit of smoking how to get rid of it?"
retrieved_chunks = retrieve(prompt, top_k=5)
context = " ".join(retrieved_chunks)

input_text = f"Context: {context}\nQuestion: {prompt}\nAnswer:"

answer = generate_response(input_text)

print(f"Q: {prompt}\nA: {answer}")


Q: I have a habit of smoking how to get rid of it?
A: Stop smoking so that I can clean the room.
I think you are exaggerating your feelings. To take a step back, let me say that every smoker experiences a deep, negative emotion about getting rid—for example, anger. As you may have already guessed, there is a real connection between smoking, cigarette addiction, depression, alcoholism and many other negative emotions. Smoking can even cause you to suffer. Cigarettes have the added bonus of increasing your risk of developing heart disease—and in many cases even causing cancer—which increases your risks of being addicted to a nicotine substitute. I don't know if you've ever experienced anxiety. No, my experience is that it is the opposite of how I feel about smoking: it's more about our


In [26]:
prompt = "i have bad habit of getting up late how to overcome it?"
retrieved_chunks = retrieve(prompt, top_k=5)
context = " ".join(retrieved_chunks)

input_text = f"Context: {context}\nQuestion: {prompt}\nAnswer:"

answer = generate_response(input_text)

print(f"Q: {prompt}\nA: {answer}")


Q: i have bad habit of getting up late how to overcome it?
A: The good news is that you don't have all of these problems to solve. There are some simple, yet powerful and powerful strategies for overcoming them. One of them is mindfulness. The practice of mindfulness has proven to help you achieve more than one thing at a time: staying engaged in daily life and living life at its highest. A quick way to try mindfulness is through your daily activities. Here are three important things you should take care of: 1. Focus on your activity 3. Take care not to lose track of time and attention 4. Never make a mistake by not engaging in one or more activities 3: Don't make mistakes by taking too many breaks on one activity 5. Don`t give up on a task and take it a step
